In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
dataset = pd.read_csv('GPUs.csv', skiprows=1, sep=';')

In [ ]:
dataset.head(30)

In [ ]:
dataset.describe()

# Promethe

In [ ]:
# Preference funcions

def pref_v_shape(d, q, p):
    if d > p:
        return 1
    elif d <= q:
        return 0
    else:
        return (d - q) / (p - q)

In [ ]:
def comprehensive_preference_index(item1, item2, pref_info):
    val = 0.0
    for i in range(len(pref_info)):
        g = pref_info.iloc[i].name
        w = pref_info.iloc[i]["w"]
        q = pref_info.iloc[i]["q"]
        p = pref_info.iloc[i]["p"]
        a = item1[g]
        b = item2[g]
        d = a - b if pref_info.iloc[i]["cost_or_gain"] == "g" else b - a
        pref = pref_info.iloc[i]["pref_func"](d, q, p)
        val += pref * w
    
    val /= pref_info["w"].sum()

    return val 

In [ ]:
pref_info = pd.DataFrame({
    "g": ["Koszt", "Wydajnosc", "TDP", "Ocena"],
    "q": [100, 15, 10, 5],
    "p": [300, 40, 30, 30],
    "w": [3, 4, 1, 2],
    "cost_or_gain": ["c", "g", "c", "g"],
    "pref_func": [pref_v_shape]*4
})

pref_info = pref_info.set_index("g")

In [ ]:
pref_info

In [ ]:
comp_table = []

for i in range(len(dataset)):
    item1 = dataset.iloc[i]
    row = []
    for j in range(len(dataset)):
        item2 = dataset.iloc[j]
        result = comprehensive_preference_index(item1, item2, pref_info)
        row.append(result)
    comp_table.append(row)

comp_df = pd.DataFrame(comp_table)
comp_df

In [ ]:
flow_plus = comp_df.sum(axis=1)
flow_minus = comp_df.sum(axis=0)
flow = flow_plus - flow_minus
flow_df = pd.DataFrame()
flow_df['Nazwa'] = dataset['Nazwa']
flow_df['flow+'] = flow_plus
flow_df['flow-'] = flow_minus
flow_df['flow'] = flow
flow_df

### Promethe I

In [ ]:
flow_df.sort_values("flow+", ascending=False)

In [ ]:
flow_df.sort_values("flow-", ascending=True)

In [ ]:
rank_table = []

for i in range(len(dataset)):
    i1_f_p = flow_df['flow+'][i]
    i1_f_m = flow_df['flow-'][i]
    row = []
    for j in range(len(dataset)):
        i2_f_p = flow_df['flow+'][j]
        i2_f_m = flow_df['flow-'][j]

        result = '' 
        if (i1_f_p > i2_f_p and i1_f_m < i2_f_m) or (i1_f_p == i2_f_p and i1_f_m < i2_f_m) or (i1_f_p > i2_f_p and i1_f_m == i2_f_m):
            result = 'P' 
        elif i1_f_p == i2_f_p and i1_f_m == i2_f_m:
            result = 'I' 
        elif (i1_f_p > i2_f_p and i1_f_m > i2_f_m) or (i1_f_p < i2_f_p and i1_f_m < i2_f_m):
            result = 'R'
        row.append(result)
    rank_table.append(row)

rank_df = pd.DataFrame(rank_table)
rank_df

In [ ]:
def draw_rank(rank_df, flow_df):
    G = nx.DiGraph()

    for i in flow_df.sort_values("flow", ascending=False).index.tolist():
        G.add_node(i)
        for j in range(len(rank_df)):
            if rank_df[j][i] == 'P' and (rank_df[j][rank_df.index[rank_df.loc[i]=='P']] == 'P').sum() == 0:
                G.add_edge(i, j)

    pos = nx.circular_layout(G)
    nx.draw(G, pos, with_labels=True, node_size=500)
    plt.title("Promethe I ranking")
    plt.show()


draw_rank(rank_df, flow_df)

### Promethe II

In [ ]:
flow_df.sort_values("flow", ascending=False)

# Electre

In [ ]:
class electree:
    def __init__(self):
        self.ranking_descending = {}
        self.ranking_ascending = {}
        self.median_ranking = {}
        self.data = None
        self.nazwa = None
        self.c = None
        self.D = None
        self.C = None
        self.sigma = None
        self.rows = None
        self.cols = None

    def load_data(self, filepath):
        data = pd.read_csv(filepath, sep=';', skiprows=1)
        pd.set_option('display.max_rows', None)
        self.data = data.rename(columns={'Unnamed: 0': 'Nazwa'})
        self.nazwa = self.data['Nazwa']
        self.data.pop('Nazwa')
        self.rows = self.data.shape[0]
        self.cols = self.data.shape[1]

    def part_c(self, p, q):
        def cost_c(a, b, p, q):
            if a - b <= q:
                return 1
            elif a - b > p:
                return 0
            return (p - (a - b)) / (p - q)
        
        def gain_c(a, b, p, q):
            if a - b >= -q:
                return 1
            elif a - b < -p:
                return 0
            return (p - (b - a))  / (p - q)
        
        matrix = np.zeros((self.rows, self.rows, self.cols))
        for i, a in enumerate(self.data.values):
            for j, b in enumerate(self.data.values):
                if i == j:
                    matrix[i, j, :] = 1
                    continue
                for idx, criterion in enumerate(a):
                    if idx in (1, 3):
                        matrix[i, j, idx] = gain_c(criterion, b[idx], p[idx], q[idx])
                    else:
                        matrix[i, j, idx] = cost_c(criterion, b[idx], p[idx], q[idx])
        self.c = matrix
        
    def part_d(self, p, v):
        def gain_d(a, b, p, v):
            if a - b <= -v:
                return 1
            elif a - b >= -p:
                return 0
            return ((b - a) - p)  / (v - p)

        def cost_d(a, b, p, v):
            if a - b >= v:
                return 1
            elif a - b <= p:
                return 0
            return (v - (a - b)) / (v - p)

        matrix = np.zeros((self.rows, self.rows, self.cols))
        for i, a in enumerate(self.data.values):
            for j, b in enumerate(self.data.values):
                if i == j:
                    continue
                for idx, criterion in enumerate(a):
                    if idx in (1, 3):
                        if self.c[i, j, idx] == 0:
                            matrix[i, j, idx] = gain_d(criterion, b[idx], p[idx], v[idx])
                        else:
                            matrix[i, j, idx] = 0
                    else:
                        if self.c[i, j, idx] == 0:
                            matrix[i, j, idx] = cost_d(criterion, b[idx], p[idx], v[idx])
                        else:
                            matrix[i, j, idx] = 0
        self.D = matrix

    def full_c(self, w):
        weighted = self.c * w
        self.C =  np.sum(weighted, axis=2) / np.sum(w)
        
    def rank_credibility(self):
        matrix = np.zeros_like(self.C)
        for i in range(self.rows):
            for j in range(self.rows):
                f = np.where(self.D[i, j, :] > self.C[i, j])
                if len(f[0]) == 0:
                    matrix[i, j] = self.C[i, j]
                else:
                    if np.any(self.D[i, j, f] == 1):
                        matrix[i, j] = 0
                    else:
                        matrix[i, j] = self.C[i, j] * np.prod((1 - self.D[i, j, f]) / (1 - self.C[i, j]))
        self.sigma = matrix

    def destilation(self, descending):
        correction = lambda x: -0.15 * x + 0.3
        no = 1
        np.fill_diagonal(self.sigma, 0)

        matrix = self.sigma.copy()
        indices = np.arange(0, self.sigma.shape[0], dtype=int)
        matrix = np.c_[matrix, indices]
        
        while matrix.size:
            # znajdź próg wiarygodności
            lambda_up = np.max(matrix[:,:-1])

            # jeżeli jest równy 0 to kończę wykonywanie algorytmu
            if lambda_up == 0:
                self.ranking_descending[no] = matrix[:,-1]
                break

            # znajdź dolny próg
            lambda_down = np.max(matrix[:,:-1][np.where(matrix[:,:-1] < lambda_up - correction(lambda_up))])

            # zapisz sigme jezeli A > B
            c1 = (matrix[:,:-1] > lambda_down)
            c2 = (matrix[:,:-1] > matrix[:,:-1].T + correction(matrix[:,:-1]))
            without_last_col = matrix[:,:-1]
            current = np.where((c1) & (c2), without_last_col, 0)

            # siła, słabość
            strength = np.count_nonzero(current, axis=1)
            weakness = np.count_nonzero(current, axis=0)
            quality = strength - weakness

            # zapisuję indeksy najlepszych wariantów
            best_alternatives = np.where(quality == np.max(quality))[0] if descending else np.where(quality == np.min(quality))[0]
            idxs = np.array(matrix[best_alternatives, -1])
        
            # destylacja wewnętrzna
            while idxs.size != 1:
                initial_destilation = matrix[best_alternatives,:-1]
                initial_destilation = initial_destilation[:, best_alternatives]

                lambda_up = lambda_down

                if lambda_up == 0:
                    break
                    
                if np.where(initial_destilation < lambda_up - correction(lambda_up))[0].size != 0:
                    lambda_down = np.max(initial_destilation[np.where(initial_destilation < lambda_up - correction(lambda_up))])
                else:
                    lambda_down = 0

                c1 = (initial_destilation > lambda_down)
                c2 = (initial_destilation > initial_destilation.T + correction(initial_destilation))
                current = np.where((c1) & (c2), initial_destilation, 0)
                strength = np.count_nonzero(initial_destilation, axis=0)
                weakness = np.count_nonzero(initial_destilation, axis=1)
                quality = strength - weakness
                best_alternatives = np.where(quality == np.max(quality))[0] if descending else np.where(quality == np.min(quality))[0]
                
                idxs = idxs[best_alternatives]

            if descending:
                self.ranking_descending[no] = idxs
            else:
                self.ranking_ascending[no] = idxs
            
            for i in idxs:
                indices = indices[indices != i]
            
            for idx in idxs:
                row_to_delete = np.where(matrix[:,-1] == idx)[0]
                for i in range(2):
                    matrix = np.delete(matrix, row_to_delete, axis=i)
            no += 1

    def ranking(self):
        keys = list(self.ranking_descending.keys())
        values = list(self.ranking_descending.values())
        for k, v in zip(keys[::-1], values):
            self.ranking_descending[k] = v
    
        graph = {}
        asc_el = self.ranking_ascending.keys()
        desc_el = self.ranking_descending.keys()
        for el in range(self.rows):
            for i in desc_el:
                if el in self.ranking_descending[i]:
                    desc_path = i
                    break
            for i in asc_el:
                if el in self.ranking_ascending[i]:
                    asc_path = i
                    break
            if desc_path + asc_path not in graph.keys():
                graph[desc_path + asc_path] = [el]
            else:
                graph[desc_path + asc_path].append(el)            
        
        graph = dict(sorted(graph.items()))
        self.median_ranking = {k+1: v for k, v in zip(range(len(graph)), list(graph.values())[::-1])}

    def print_ranking(self, which_ranking):
            res = ''
            if which_ranking == 'median':
                print('Ranking medianowy')
                parts = []
                for k, ls in self.median_ranking.items():
                    names = [self.nazwa[i] for i in ls]
                    a = ', '.join([str(el) for el in names])
                    parts.append(a)
                for i, part in enumerate(parts):
                    res += f'{i+1}. {part}\n'
            if which_ranking == 'asc':
                print('Ranking wstępujący')
                parts = []
                for k, ls in self.ranking_ascending.items():
                    names = [self.nazwa[i] for i in ls]
                    a = ', '.join([str(el) for el in names])
                    parts.append(a)
                for i, part in enumerate(parts[::-1]):
                    res += f'{i+1}. {part}\n'
            if which_ranking == 'desc':
                print('Ranking zstępujący')
                parts = []
                for k, ls in self.ranking_descending.items():
                    names = [self.nazwa[i] for i in ls]
                    a = ', '.join([str(el) for el in names])
                    parts.append(a)
                for i, part in enumerate(parts[::-1]):
                    res += f'{i+1}. {part}\n'

            print(res)


In [ ]:
p = [300, 40, 30, 30]
q = [100, 15, 10, 5]
v = [700, 50, 50, 50]
w = [3, 4, 1, 2]

electre = electree()
electre.load_data('GPUs.csv')
electre.part_c(p, q)
electre.part_d(p, v)
electre.full_c(w)
electre.rank_credibility()
electre.destilation(True)
electre.destilation(False)
electre.ranking()

# electre.print_ranking('median')
electre.print_ranking('asc')
# electre.print_ranking('desc')